# Лабораторна №1

## Завдання 

Реалізувати тип "множина" 

Реалізувати базові операції:<br>
- Insert [+]
- Delete [+]
- Search [+]
- Clear [+]


Реалізувати додаткові операції:<br>
- Union [+]
- Intersection [+]
- SetDifference [+]
- SymDifference [+]
- isSubset [+]


Провести експериментальні заміри часу операцій 
- Search
- Union

## Код

In [42]:
class BitVectorSet:
    def __init__(self, t):
        """Ініціалізує множину з t 64-бітних регістрів (універсум з 64t елементів)."""
        self.t = t 
        self.vector = [0] * t

    def _get_index(self, element):
        """Повертає індекс регістра та позицію біта в цьому регстрі"""
        register_index = element >> 6 # Індекс 64-бітного регістра math.log2(64)
        bit_position = element & (64-1) # Позиція біта в регістрі
        return register_index, bit_position
    
    def have(self, element):
        """Перевіряє чи існує елемент в множині"""
        reg_index, b_position = self._get_index(element)
        return self.vector[reg_index] & (1 << b_position) != 0
        
    def add(self, element):
        """Додає елемент в множину"""
        reg_index, b_position = self._get_index(element)
        self.vector[reg_index] |= 1 << b_position

    def delete(self, element):
        """Видаляє елемент з множини"""
        reg_index, b_position = self._get_index(element)
        self.vector[reg_index] &= ~(1 << b_position)

    def union(self, other):
        """Повертає обєднання двух множин"""
        if self.t != other.t:
            raise ValueError("Множини мають різні розміри")
        result = BitVectorSet(self.t)
        result.vector = [self.vector[i] | other.vector[i] for i in range(self.t)]
        return result
    
    def intersection(self, other):
        """Повертає перетин двух множин"""
        if self.t != other.t:
            raise ValueError("Множини мають різні розміри")
        result = BitVectorSet(self.t)
        result.vector = [self.vector[i] & other.vector[i] for i in range(self.t)]
        return result
    
    def diff(self, other):
        """Повертає різницю двух множин"""
        result = BitVectorSet(self.t)
        result.vector = [self.vector[i] & ~other.vector[i] for i in range(self.t)]
        return result
    
    def sym_diff(self, other):
        """Повертає симетричну різницю: елементи, які є в одному з множин, але не в обох."""
        if self.t != other.t:
            raise ValueError("Множини мають різні розміри")
        result = BitVectorSet(self.t)
        result.vector = [self.vector[i] ^ other.vector[i] for i in range(self.t)]
        return result
    
    def issubset(self, other):
        """Перевіряє, чи є поточна множина підмножиною іншої."""
        if self.t != other.t:
            raise ValueError("Множини мають різні розміри")
        # Для кожного регістра перевіряємо, чи всі встановлені біти self є в other
        for i in range(self.t):
            if self.vector[i] & ~other.vector[i] != 0: 
                return False
        return True
    
    def clear(self):
        """Очищає множину, встановлюючи всі біти в 0."""
        self.vector = [0] * self.t
    
    def __str__(self):
        """Повертає бітовий вектор як рядок для зручного перегляду."""
        return ''.join(f'{reg:064b}' for reg in reversed(self.vector))


## Приклад роботи

In [43]:
A = BitVectorSet(2)
B = BitVectorSet(2)

In [44]:
print(A)
print(B)

00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000


In [45]:
A.add(127)
B.add(127)
A.add(0)
B.add(1)
A.add(3)
B.add(4)

In [46]:
print(A)
print(B)

10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001001
10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010010


In [48]:
A.have(127), B.have(0)

(True, False)

In [52]:
A.delete(3)
B.delete(4)

In [57]:
print(A)
print(B)

10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001
10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010


In [53]:
print(A.union(B))

10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000011


In [54]:
print(A.intersection(B))

10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000


In [55]:
print(A.diff(B))

00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001


In [56]:
print(A.sym_diff(B))

00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000011


In [58]:
print(A.issubset(B))

False


In [59]:
A.clear()

In [60]:
print(A)

00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000


## Заміри часу

In [74]:
import random
import time

In [116]:
def genranset(t: int, shouldexist = "dont_need", shouldnotexist = "dont_need"):
    """Генерує випадкову множину.

    t: розмір множини
    shouldexist: елемент який точно повинен існувати
    shouldntexist: елемент який точно не повинен існувати
    """
    S = BitVectorSet(t)

    for i in range(64 * t):  # Ітерація по всіх елементах універсуму
        if random.choice([True, False]):  # Випадково вирішуємо, чи додавати елемент
            S.add(i)

    if type(shouldexist) == int and 0 <= shouldexist < 64 * t: 
        S.add(shouldexist) # Додаємо елемент який повинен існувати

    if type(shouldnotexist) == int and 0 <= shouldnotexist < 64 * t:
        S.delete(shouldnotexist) # Додаємо елемент який не повинен існувати
        
    return S

In [117]:
def timeSearchElementExist(t):
    """Рахує час для перевірки існування елементу, який точно існує.
    
    t: розмір множини
    """
    element = random.randint(0, 64 * t - 1)  # Генеруємо піддослідного кролика
    ranset = genranset(t, shouldexist=element)  # Генеруємо множину з піддослідним кроликом
    
    start_time = time.time()  # Початок часу
    ranset.have(element)  # Перевіряємо, чи існує елемент в множині
    end_time = time.time()  # Кінець часу
    
    elapsed_time = end_time - start_time  # Різниця в часі (в секундах)
    return elapsed_time


In [118]:
def timeSearchElementNotExist(t):
    """Рахує час для перевірки існування елементу, який точно не існує.
    
    t: розмір множини
    """
    element = random.randint(0, 64 * t - 1)  # Генеруємо піддослідного кролика
    ranset = genranset(t, shouldnotexist=element)  # Генеруємо множину без піддослідного кролика
    
    start_time = time.time()  # Початок часу
    ranset.have(element)  # Перевіряємо, чи існує елемент в множині
    end_time = time.time()  # Кінець часу
    
    elapsed_time = end_time - start_time  # Різниця в часі (в секундах)
    return elapsed_time

In [119]:
def timeUnion(t):
    """Рахує час для обєднання двух множин.
    
    t: розмір множин
    """
    ranset1 = genranset(t)
    ranset2 = genranset(t)

    start_time = time.time()  # Початок часу
    ranset1.union(ranset2)  # Перевіряємо, чи існує елемент в множині
    end_time = time.time()  # Кінець часу

    elapsed_time = end_time - start_time  # Різниця в часі (в секундах)
    return elapsed_time

In [120]:
def timeControl(t, n, type):
    """Функція для контролю часу виконання операцій з множиною.
    
    t: розмір множини
    n: кількість повторів
    type: тип операції ('searchexist', 'searchnotexist', 'union')
    """
    # Перевірка типу операції
    if type not in ['searchexist', 'searchnotexist', 'union']:
        raise ValueError("Тип повинен бути 'searchexist', 'searchnotexist' або 'union'")
    
    sum_time = 0 
    match type:
        case 'searchexist':
            for i in range(n):
                i_time = timeSearchElementExist(t)
                sum_time += i_time
        case 'searchnotexist':
            for i in range(n):
                i_time = timeSearchElementNotExist(t)
                sum_time += i_time
        case 'union':
            for i in range(n):
                i_time = timeUnion(t)
                sum_time += i_time
    avg_time = sum_time/n
    return avg_time


In [123]:
timeControl(1000, 1000, 'searchexist')

1.0673999786376954e-06

In [124]:
timeControl(1000, 1000, 'searchnotexist')

4.547357559204102e-06

In [105]:
timeControl(1000, 1000, 'union')

0.00013441014289855956